In [ ]:
import math
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from scipy.stats import norm
import torch
import matplotlib.pyplot as plt
import os
import time
import options as options
import distributions as distributions
import nn_utils as nn_ut
import utils as ut

In [ ]:
# defining the payoff function
K = 1.
# uncomment desired option and output name
option = options.MaxCall(K)
output_name = 'maxcall_multi'
# option = options.BasketCall(K)
# output_name = 'basketcall_multi'
# option = options.MinPut(K)
# output_name = 'minput_multi'
# option = options.GeometricPut(K)
# output_name = 'geomput_multi'

p = 3
t = 1.
h = 1. / 12
cost = nn_ut.PowerCost(p, h, 'second order')
d_list = list(range(1,21))

In [ ]:
### parameters for neural network and training

# fixing the seed
torch.manual_seed(29)

# defining the risk measure object
width = 20
depth = 4
sample_size = 100
learn_rate = 0.001

roll_window = 100
epochs = 10000
df_train = pd.DataFrame(index=np.arange(roll_window, epochs), columns=d_list)
df_results = pd.DataFrame(index=pd.Index(d_list), columns=['assets', 't0', 'tfin', 'expected', 'expected_mc_err', 'worst_case', 'mc_err', 'epochs', 'train_time'])

In [ ]:

for d in d_list:
    
    out_dict = {}
    out_dict['assets'] = d
    out_dict['t0'] = t
    out_dict['tfin'] = t + h
    out_dict['epochs'] = epochs

    Sigma = 0.20 * torch.eye(d) # lower triangular square root of the covariance matrix
    mu = distributions.MultiLogNormal(loc=- 0.5 * t * torch.sum(torch.pow(Sigma, 2), dim=1), scale_tril=math.sqrt(t) * Sigma)

    risk_measure = nn_ut.MartRiskMeasureMulti(option.f, cost.cost, mu, torch.nn.ReLU, width, depth, d=d)

    # otpimizer
    optim = torch.optim.Adam(risk_measure.parameters(), lr=learn_rate)

    # training cycle
    start_training = time.time()
    train_hist = []
    for i in range(epochs):
        optim.zero_grad()
        y = mu.sample([sample_size])
        risk = risk_measure(y)
        risk.backward()
        optim.step()
        train_hist.append(- float(risk.detach()))

    # saving training results and statistics
    out_dict['train_time'] = time.time() - start_training

    mc_samples = 100000
    final_samples = mu.sample([mc_samples])
    out_dict['expected'] = torch.mean(option.f(final_samples)).item()
    out_dict['expected_mc_err'] = 2.33 * torch.std(option.f(final_samples)).item() / math.sqrt(mc_samples)

    risk_measure.eval()
    out_dict['worst_case'] = risk_measure(final_samples).item()
    out_dict['mc_err'] = risk_measure.mc_err.item()

    df_results.loc[d] = out_dict

    # saving training history
    train_roll = pd.Series(train_hist).rolling(roll_window).mean().dropna()
    df_train[d] = train_roll


In [ ]:
# saving an excel file with the results
ut.check_dir("output")
excel_file = os.path.join('output', 'geomput_multi' + '.xlsx')

with pd.ExcelWriter(excel_file, engine='openpyxl') as writer:
    df_results.to_excel(writer, sheet_name='Results', index=False)
    df_train.to_excel(writer, sheet_name='Training', index=False)

In [ ]:
# plotting the results
fig, ax1 = plt.subplots()

ax1.set_xlabel('Number of assets')
ax1.set_ylabel('Fair Value', color='tab:green')
ax1.plot(df_results['assets'], df_results['expected'], color='tab:green', label='reference model')
ax1.plot(df_results['assets'], df_results['worst_case'], color='tab:red', label='upper bound')
ax1.tick_params(axis='y', labelcolor='tab:green')
ax1.set_xticks(np.arange(df_results['assets'][1], df_results['assets'][df_results.shape[0]], 1.0))

# Create a twin Axes sharing the xaxis for the second quantity
ax2 = ax1.twinx()
color = 'tab:blue'
ax2.set_ylabel('Computational Time (s)', color=color)
ax2.plot(df_results['assets'], df_results['train_time'], linestyle='-.', color=color)
ax2.set_ylim([0,7])
plt.tick_params(axis='y', labelcolor=color)

# Combine legends from both axes
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc='lower right')
plt.show()